In [64]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

from __future__ import division

import numpy as np
import os
#import mne
import pandas as pd
import matplotlib.pyplot as plt

from scipy import io, signal

from data_utils import *
from preprocessing import *
from models import *
from metrics import *

from sklearn.linear_model import LogisticRegression as LR
from sklearn import svm

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
for key,value in sorted(ch_dict.items(), key=lambda x: x[1]):
    print key,value,'\t',

FP1 0 	FPZ 1 	FP2 2 	AF3 3 	AF4 4 	F7 5 	F5 6 	F3 7 	F1 8 	FZ 9 	F2 10 	F4 11 	F6 12 	F8 13 	FT7 14 	FC5 15 	FC3 16 	FC1 17 	FCZ 18 	FC2 19 	FC4 20 	FC6 21 	FT8 22 	T7 23 	C5 24 	C3 25 	C1 26 	CZ 27 	C2 28 	C4 29 	C6 30 	T8 31 	M1 32 	TP7 33 	CP5 34 	CP3 35 	CP1 36 	CPZ 37 	CP2 38 	CP4 39 	CP6 40 	TP8 41 	M2 42 	P7 43 	P5 44 	P3 45 	P1 46 	PZ 47 	P2 48 	P4 49 	P6 50 	P8 51 	PO7 52 	PO5 53 	PO3 54 	POZ 55 	PO4 56 	PO6 57 	PO8 58 	CB1 59 	O1 60 	OZ 61 	O2 62 	CB2 63 	VEO 64 	HEO 65 	EKG 66 	EMG 67 	Trigger 68 	STI 014 69 	

In [72]:
X_s, Y_s, Y_l, ch_dict = import_data(1, downsample=5,
                                     ch_dict=True, bp_range=(0.5,40))

Converting time column to int64...


In [108]:
# state vs rest
subj = 0
X, Y = X_s[subj], Y_s[subj]
#Y = narrow(Y, 4, .2, 1)

if False:
    channs = 'FC6 FT8 C4 T7 C5 C3 CP5 CP3 CP1 P3'.split()
    channs = map(lambda ch: ch_dict[ch], channs)
    X = X[:, channs]
elif False:
    channs = range(64)
    channs.remove(32)
    channs.remove(42) #remove m1, m2
    X = X[:, channs]
elif False:
    X = X[:,67][:,None]
print "number of channels used:", X.shape[1]

fs = 200
window_size = 2000
window_stride = 500
num_wins = 5
# choose labels to compare
label = 3
rest = 1

# locate labels
Y_1 = (Y == label).astype(int)
Y_0 = (Y == rest).astype(int)
diff0, diff1 = np.diff(Y_0), np.diff(Y_1)
# compute window limits
start0, end0 = map(lambda x: np.where(x)[0], [diff0>0, diff0<0])
start1, end1 = map(lambda x: np.where(x)[0], [diff1>0, diff1<0])

mapper = lambda lims: X[lims[0]:lims[1]]
X0, X1 = rec_map(mapper, [zip(start0, end0), zip(start1, end1)], 2)
# window series
mapper = lambda win: window_series(win, width=window_size, 
                                   stride=window_stride, fs=fs, num_wins=num_wins)
X0, X1 = rec_map(mapper, [X0, X1], 2)
# flatten windows
mapper = lambda x: [i for j in x for i in j]
X0, X1 = map(mapper, [X0, X1])
# split data
X0_train, X0_test = split(X0)
X1_train, X1_test = split(X1)
X_train, X_test = map(np.concatenate, [[X0_train, X1_train], [X0_test, X1_test]])
# create label vectors
Y0_train, Y0_test = np.zeros(len(X0_train)), np.zeros(len(X0_test))
Y1_train, Y1_test = np.ones(len(X1_train)), np.ones(len(X1_test))
Y_train, Y_test = map(np.concatenate, [[Y0_train, Y1_train], [Y0_test, Y1_test]])
# verify dimensions
dim(X_train)
dim(X_test)

print "train:test :", len(X_train), len(X_test)



(X3_train, Y3_train), (X3_test, Y3_test) = (X_train, Y_train), (X_test, Y_test)

bal = True
if bal:
    (X3_train, Y3_train), (X3_test, Y3_test) = map(balance, (X3_train, X3_test), (Y3_train, Y3_test))
    #_, (X3_test, Y3_test) = map(balance, (X3_train, X3_test), (Y3_train, Y3_test))

print "ratio of true:false : ", sum(list(Y3_train) + list(Y3_test))/(len(Y3_train)+len(Y3_test))
print "N train:{} N test:{}".format(len(X3_train),len(X3_test))
#mapper = lambda wins: feature_vect(wins, 5, norm=True)

ftype = 1
norm = True


X3_train, means, stds = feature_vect(X3_train, ftype, trim=False, norm=norm)
X3_test, _, _ = feature_vect(X3_test, ftype, trim=False, norm=norm)

number of channels used: 62
1104 62 400 
546 62 400 
train:test : 1104 546
ratio of true:false :  0.5
N train:1104 N test:546
(1104, 2542)
(546, 2542)


In [109]:
# model fit
if False:
    mapper = lambda x: x[mu_keeps]
    X3_train, X3_test = rec_map(mapper, [X3_train, X3_test], 2)

X_tr, Y_tr = X3_train, Y3_train
X_ts, Y_ts = X3_test, Y3_test

mtype = 'LR'
if mtype == 'LR':
    model = LR(n_jobs=-1)
elif mtype == 'SVM':
    model = svm.SVC(cache_size=1000)
elif mtype == 'linSVM':
    model = svm.LinearSVC()

print "number of features:", len(X3_train[0])

model.fit(np.asarray(X_tr), np.asarray(Y_tr))

Y_pr = model.predict(np.asarray(X_ts))
yy = model.predict(np.asarray(X_tr))
print "fit"
metrics(Y_tr, yy)
print "predict"
metrics(Y_ts, Y_pr)

number of features: 2542
fit
accuracy: 1.0
precision: 1.0
recall 1.0
predict
accuracy: 0.771062271062
precision: 0.768115942029
recall 0.776556776557


In [ ]:
#plt.plot(np.std(X3_train, axis=0))
#plt.figure(2), plt.plot(stds)
#print np.shape(means), np.shape(np.mean(X3_train, axis=0))
#print sum(np.std(X3_train, axis=0)==0.0)

In [ ]:
keeps = (stds==0.0)
print sum(keeps)


mapper = lambda v: v[keeps]
keeps2 = np.std(map(mapper, X3_train), axis=0)
print sum(keeps2)

In [ ]:
### TSNE visualization

from sklearn.manifold import TSNE
if False:
    model = TSNE()
    Xd = model.fit_transform(map(lambda x: x.flatten(), X_train))
    plt.scatter(*Xd.T, c=Y_train, cmap='gray')
    
if False:
    model = TSNE()
    Xd = model.fit_transform(X3_train)
    plt.scatter(*Xd.T, c=Y3_train, cmap='gray')

In [ ]:
xx0, yy0 = map(np.asarray, [X3_train, Y3_train])
xx1, yy1 = map(np.asarray, [X3_test, Y3_test])

In [ ]:
plt.plot(np.mean(xx0, axis=0))

In [ ]:

x0, x1 = xx0[yy0==0], xx0[yy0==1]

m00, m01 = x0.mean(0), x1.mean(0)

plt.plot(m00)
plt.plot(m01)                           

###

plt.figure(2)



x0, x1 = xx1[yy1==0], xx1[yy1==1]

m10, m11 = x0.mean(0), x1.mean(0)

plt.plot(m10)
plt.plot(m11)  

plt.figure(3)

for i,m in enumerate([m00, m01, m10, m11]):
    plt.plot(m[:100], label=str(i))
    
plt.legend()

plt.figure(4)

plt.plot(m00-m01)
plt.plot(m10-m11)

In [ ]:
mu_diff = (m10-m11)-(m00-m01)
plt.plot(mu_diff)
mu_keeps = np.sign(m11-m10) == np.sign(m01-m00)

plt.plot(np.sort(np.abs(mu_diff)))
plt.plot(mu_keep.astype(int))

In [ ]:
plt.plot(mu_keep[:500])

In [ ]:
print x0.shape, x1.shape

In [ ]:
_ = plt.plot(np.mean(X3_train, axis=0))
_ = plt.plot(np.mean(X3_test, axis=0))

plt.figure(2)
_ = plt.plot(X3_test[2])

In [ ]:
plt.plot(Y3_pred)
plt.plot(Y3_test)

In [ ]:
np.sign(-32)

In [ ]:
xf = signal.spectrogram(X_s[0].T, fs=200)

In [ ]:
xf = xf[2]

In [ ]:
xf.shape

In [ ]:
X_s[0].shape

In [ ]:
plt.imshow(xf[3,:20,:], aspect='auto', interpolation='none')
plt.colorbar()

In [ ]:
X2_s, _, _, _ = import_data(1, downsample=5, 
                                     ch_dict=True, bp_range=None)

In [ ]:
x2f = signal.spectrogram(X2_s[0].T, nperseg=200,fs=200)[2]

In [ ]:
plt.imshow(x2f[3,:20,800:1000], aspect='auto', interpolation='none')
plt.colorbar()

In [ ]:
plt.imshow(X3_test, aspect='auto')


In [ ]:
dim(X3_test)

In [ ]:
2542%62

In [ ]:
plt.imshow(X3_train, aspect='auto')